In [1]:
import os
import glob
import re
import pandas as pd

In [10]:
directory = "/nesi/nobackup/uoa04099/KOLF2_ont-str-and-sv/"
my_path = directory + "STRs/KOLF2_T13_base_mods-5mC.sorted.aligned/KOLF2_T13_base_mods-5mC.sorted.aligned.details"
print(my_path)
os.chdir(my_path)

straglr_gz = "/nesi/nobackup/uoa04099/KOLF2_update_repeats/KOLF2.wf_str.vcf"
straglr_in = os.system("gunzip " + straglr_gz)



/nesi/nobackup/uoa04099/KOLF2_ont-str-and-sv/STRs/KOLF2_T13_base_mods-5mC.sorted.aligned/KOLF2_T13_base_mods-5mC.sorted.aligned.details


In [11]:
# Method one: shell command to concatenate each summary file
os.system("ls ch*/*summary.txt > summary_concat.txt")


0

In [12]:
#Read in and merge NanoRepeat output

output = []

for file in glob.glob('chr*/*summary.txt'):
    df = pd.read_csv(file, sep='\t', header=None).T
    temp_df = df[0].str.split('=', expand=True).set_index(0)
    temp_df.to_csv("/nesi/nobackup/uoa04099/temp_df.txt", sep="\t")
    output.append(temp_df)
    
outdf = pd.concat(output, axis=1).T
locus_str = outdf.iloc[:,1].str.split("-", expand=True)
                                      
all_df = [locus_str, outdf.iloc[:,3:4], outdf.iloc[:,6:7],outdf.iloc[:,8]]
nano = pd.concat(all_df, axis=1)

nano.columns = ["#chr", "start", "end", "motif", "num_alleles", "Allele_1_Repeat_Size", "Allele_2_Repeat_Size"]
nano["#chr"] = nano["#chr"].str.replace('X','100').str.replace(r'\D*','', regex=True).astype(int).sort_values()
nano["start"] = nano["start"].astype(int)
nano = nano.sort_values(["#chr", "start"])
nano["#chr"] = nano["#chr"].astype(str).replace('100','X')
#print(nano)

nano.set_index("#chr").to_csv("/nesi/nobackup/uoa04099/nanorepeat_summarised.txt", sep="\t")
                    

In [13]:
#Read in and merge straglr output

colnames=names=['chr','pos','id','ref', 'alt','qual','filter','info','format','kolf2']
straglr = pd.read_csv(straglr_in, sep='\t', comment='#', names=colnames)


straglr["chr"] =straglr["chr"].str.replace('X','100').str.replace(r'\D*','', regex=True).astype(int).sort_values()
straglr["pos"] = straglr["pos"].astype(int)
straglr = straglr.sort_values(["chr", "pos"])
straglr["chr"] = straglr["chr"].astype(str).replace('100','X')
print(straglr)



ValueError: Invalid file path or buffer object type: <class 'int'>

In [102]:
len(nano)

48

In [103]:
len(straglr)

47